# Read data from weather mast at Haukeliseter site

## create temperature file for apriori guess
## calculate daily snowfall

In [ ]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import numpy as np 
import csv
import pandas as pd
import datetime
from datetime import date
import calendar
import get_Haukeli_obs_data as obsDat
import createFolder as cF
import netCDF4
import calc_station_properties as cs




%matplotlib inline    



In [ ]:
txt_save_dir = '../../Data/sfc_temp/txt'
cF.createFolder(txt_save_dir)

nc_save_dir = '../../Data/sfc_temp/nc'
cF.createFolder(nc_save_dir)

In [ ]:
Haukeli = pd.read_csv('../../Data/MetNo_obs/201612_Haukeliseter.txt',\
                    sep = ',',header=22, usecols = [0,1,2,3,4,6,7,8, 14], \
                      names = ['Date','TimeStamp', 'RA1', 'RA2', 'RA3', 'X2RA1', 'X2RA2', 'X2RA3', 'TA'])

In [ ]:
dd = Haukeli['Date']
TimeStamp = Haukeli['TimeStamp']     # Time Stamp
RA1 = Haukeli['RA1'].astype(float)            # total accumulation from Geonor inside DOUBLE FENCE [mm] RA1
RA2 = Haukeli['RA2'].astype(float)
RA3 = Haukeli['RA3'].astype(float)
X2RA1 = Haukeli['X2RA1'].astype(float)          # Total accumulation from Geonor South, shielded with SINGLE FENCE [mm] X2RA1
X2RA2 = Haukeli['X2RA2'].astype(float)
X2RA3 = Haukeli['X2RA3'].astype(float)
TA = Haukeli['TA'].astype(float)            # Air temperature, PT100 [deg C] TA
#wind45 = Hauke['3DFFL4']      # wind speed 4.5m @ mast 2 [m/s] 3DFFL4

In [ ]:
### exclude missing values
RA1 = RA1.where(RA1 != -999.00)
RA2 = RA2.where(RA2 != -999.00)
RA3 = RA3.where(RA3 != -999.00)
X2RA1 = X2RA1.where(X2RA1 != -999.00)
X2RA2 = X2RA2.where(X2RA2 != -999.00)
X2RA3 = X2RA3.where(X2RA3 != -999.00)
TA = TA.where(TA != -999.00)
#wind45 = wind45.where(wind45 != -999.00)

In [ ]:
# --------- connect values from double fence and calculate mean  -------------------------------------------------------------------------
# 
RA = pd.concat([RA1, RA2, RA3], axis = 1)
RA = np.nanmean(RA, axis = 1)

In [ ]:
yr = []
mm = []
dy = []
tt = []
df1 = []
df2 = []
df3 = []
df = []
temp = []
for i in range(0,31):
    idx = datetime.datetime.strptime(str(dd[i*1440]), '%Y%m%d')
    yr.append(int(idx.year))
    mm.append(int(idx.month))
    dy.append(int(idx.day))
    tt.append(TimeStamp[ i*1440 : (i+1)*1440 ])
    df1.append(RA1[i*1440: (i+1)*1440])
    df2.append(RA2[i*1440: (i+1)*1440])
    df3.append(RA3[i*1440: (i+1)*1440])
    df.append(RA[i*1440: (i+1)*1440])
    temp.append(TA[i*1440: (i+1)*1440])

df1 = np.transpose(df1)
df2 = np.transpose(df2)
df3 = np.transpose(df3)
df = np.transpose(df)
temp = np.transpose(temp)

idx = []
t_minute = []
for k in range(0,60):
    idx.append(k*100)
t_minute = idx
for h in range(1,24):
    t_minute = np.concatenate([t_minute, np.asarray(idx)+h*10000])


fmt = '%1.f %2.5f'
#fmt = '%2.5f'
#fmt = '%1.1f'

for i in range(0,31):
    if dy[i] < 10:
        day = '0%s' %dy[i]
    else:
        day = str(dy[i])
    filename = '%s/Haukeli_sfc_temp_%s%s%s.txt' %(txt_save_dir,yr[i],mm[i],day)
    np.savetxt(filename, np.transpose([t_minute,temp[:,i]]), delimiter = ',', fmt = fmt, header = 'time, sfc_temp')

for i in range(0,31):
### write netCDF file including t_minute, temp
    var = temp
    mask = np.ma.getmaskarray(var[:,i])
    var = np.ma.array(var[:,i], mask = mask, fill_value = -9999.)
    var = var.filled()
    
    if dy[i] < 10:
        day = '0%s' %dy[i]
    else:
        day = str(dy[i])
    f = netCDF4.Dataset('%s/Haukeli_sfc_temp_%s%s%s.nc' %(nc_save_dir,yr[i],mm[i],day), 'w')
### create dimensions
    f.createDimension('time', t_minute.shape[0])

    tid = f.createVariable('time', t_minute.dtype,'time',zlib = True)
    tid[:] = t_minute[:]


    dim = ('time', )
    sfc_T = f.createVariable(varname='sfc_temp', datatype=var.dtype, dimensions=dim,
                                      fill_value = -9999., zlib=True)
    sfc_T[:] = var
    f.close()